## OCR with AZ Computer Vision API  <a name="RecognizeText"> </a>
* [Detect and extract printed text from an image](#OCR)
* [Detect and extract handwritten text from an image](#RecognizeText)

## Optical character recognition (OCR) with Computer Vision API <a name="OCR"> </a>

Use the [Optical Character Recognition (OCR) method](https://westcentralus.dev.cognitive.microsoft.com/docs/services/5adf991815e1060e6355ad44/operations/56f91f2e778daf14a499e1fc) to synchronously detect printed text in an image and extract recognized characters into a machine-usable character stream.

To illustrate the OCR API, set `image_path` to point to the text to be recognized.

In [1]:
# IMPORTING THE LIBRARIES
import requests
%matplotlib inline
from PIL import Image
import matplotlib.patches as patches
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import time
from matplotlib.patches import Polygon

# PROXIES IF YOU ARE RUNNING THE JUPYTER NOTEBOOK FROM YOUR LAPTOP
proxy = "http://www.com:8080"
proxys = "https://www.com:8080"
proxyDict = { "http"  : proxy, "https"  : proxys }

In [ ]:
subscription_key = ''
assert subscription_key

image_path = "FT_OCR2.PNG"


In [ ]:
vision_base_url = "https://PYTHONCOMPUTERVISION.cognitiveservices.azure.com/vision/v1.0/"
ocr_url = vision_base_url + "ocr"
print(ocr_url)

image_data = open(image_path, "rb").read()
headers    = {'Ocp-Apim-Subscription-Key': subscription_key, 
              "Content-Type": "application/octet-stream" }
#params   = {'mode' : 'Handwritten'}
params   = {'language': 'unk', 'detectOrientation ': 'true'}
#data     = {'url': image_url}
response = requests.post(ocr_url, headers=headers, params=params, data=image_data ) #, proxies=proxyDict)
response.raise_for_status()
analysis = response.json()

Scanning the whole page

In [ ]:

line_infos = [region["lines"] for region in analysis["regions"]]
word_infos = []
for line in line_infos:
    for word_metadata in line:
        for word_info in word_metadata["words"]:
            word_infos.append(word_info)
word_infos

# PLOTTING RECTANGLES ON RECOGNIZED TEXT

plt.figure(figsize=(25,25))

image  = Image.open(image_path)
ax     = plt.imshow(image, alpha=0.5)
for word in word_infos:
    bbox = [int(num) for num in word["boundingBox"].split(",")]
    text = word["text"]
    origin = (bbox[0], bbox[1])
    patch  = Rectangle(origin, bbox[2], bbox[3], fill=False, linewidth=2, color='y')
    ax.axes.add_patch(patch)
    plt.text(origin[0], origin[1], text, fontsize=10, weight="bold", va="top")
_ = plt.axis("off")

In [ ]:
#FUTURE PANDAS PARSING
#import pandas as pd
#from pandas.io.json import json_normalize
#pd.set_option('display.max_colwidth', -1)
#df = pd.DataFrame(analysis)
#df2 = pd.DataFrame.from_dict(json_normalize(df['regions']), orient='columns')
#df2

function for field extraction

In [ ]:
# THIS FUNCTION WILL FIND TEXT AFTER AND TAG VIA RETANGLE
def print_text_box(line_infos,searched_text,regex_extraction):
    import re
    boundingBox = ''
    word_infos_found = []
    for line in line_infos:
        for word_metadata in line:
            for word_info in word_metadata["words"]:
                if searched_text in word_info['text'] :
                    boundingBox = word_info['boundingBox']  
                    #print (word_info)
                    x1s,y1s,x2s,y2s = int(boundingBox.split(',')[0]) , int(boundingBox.split(',')[1]), int(boundingBox.split(',')[2]) ,int(boundingBox.split(',')[3])

    abs_avg = y1s + int(y2s/2) 
    #print ("Searching in height:",x1s,y1s,x2s,y2s,"AVG",abs_avg )
    
    word_infos_found = []
    
    for line in line_infos:
        for word_metadata in line:
            for word_info in word_metadata["words"]:
                actual_bb = word_info['boundingBox']
                x1,y1,x2,y2 = int(actual_bb.split(',')[0]) , int(actual_bb.split(',')[1]), int(actual_bb.split(',')[2]) ,int(actual_bb.split(',')[3])
                 #int (abs( int(y1s) - int(y2s)) / 2 + int(min(y1s,y2s)))
                if ( y1 < int(abs_avg) and ( y1 + y2 ) > int(abs_avg)  ):
                    #print (x1s,y1s,x2s,y2s,"-->",abs_avg,"----->",word_info)
                    word_infos_found.append(word_info)
    
    word_infos_found_sorted = sorted(word_infos_found, key=lambda k: int(k['boundingBox'].split(',')[0])) 
    #for word_info in word_metadata["words"]:
    total_line = ""
    for item in word_infos_found_sorted:
        total_line = total_line + ' ' + item['text']
    print ("line:",total_line)
    p = re.compile(regex_extraction)
    result = p.search(total_line)
    print("field value:",result.group(1) )
    
    return word_infos_found,result.group(1)



Doing a rectangle on the field extracted

In [ ]:
word_infos_found = []

text2search = 'Equipment'
regex_for_text = "Equipment ID ..(........)"

word_infos_found,field_value_found = print_text_box(line_infos,text2search,regex_for_text)      

print ("Anchor text:",text2search,"|Regex for extraction",regex_for_text)
print ("Field value extracted:",field_value_found)
                
# PLOTTING RECTANGLES ON RECOGNIZED TEXT SEARCHED
plt.figure(figsize=(25,25))

image  = Image.open(image_path)
ax     = plt.imshow(image, alpha=0.5)
for word in word_infos_found:
    bbox = [int(num) for num in word["boundingBox"].split(",")]
    text = word["text"]
    if field_value_found in text:
        origin = (bbox[0], bbox[1])
        patch  = Rectangle(origin, bbox[2], bbox[3], fill=False, linewidth=2, color='r')
    else:
        origin = (bbox[0], bbox[1])
        patch  = Rectangle(origin, bbox[2], bbox[3], fill=False, linewidth=2, color='y')
    ax.axes.add_patch(patch)
    plt.text(origin[0], origin[1], text, fontsize=10, weight="bold", va="top")
_ = plt.axis("off")

